In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import timm
from EFF3D.MRI import *
from fastai.vision.all import *
from EFF3D.res import resnet10
import torch.nn as nn

In [ ]:
df=pd.read_csv('../nonorm/ADNCold.csv')
# path=Path('/home/staff/xin/Downloads/newMRI/ADtrain')

In [ ]:
db = DataBlock(blocks=(TransformBlock(type_tfms=partial(MriTensorImage.create)),CategoryBlock),
               get_x=ColReader('name'),
               get_y=ColReader('label'),
               splitter=RandomSplitter(valid_pct=0.2,seed=2),
              )

dls=db.dataloaders(source=df, bs=8, num_workers=4)

In [ ]:
encoder=resnet10(shortcut_type='B')
encoder=nn.DataParallel(encoder)
weight=torch.load('../nonorm/models/resnet_10_23dataset.pth')
encoder.load_state_dict(weight['state_dict'],strict=False)
encoder=nn.Sequential(*list(encoder.module.children()))
head=nn.Sequential(nn.AdaptiveMaxPool3d(1), Flatten(), nn.Linear(512, 2))
apply_init(head)
model=nn.Sequential(encoder,head)

/home/staff/xin/Documents/fastai2/SMRIEFF3D/EFF3D/res.py:174: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


In [ ]:
class SAM(Callback):
    "Sharpness-Aware Minimization"
    def __init__(self, zero_grad=True, rho=0.05, eps=1e-12, **kwargs): 
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"
        self.state = defaultdict(dict)
        store_attr()

    def params(self): return self.learn.opt.all_params(with_grad=True)
    def _grad_norm(self): 
        print(self.params())
        return torch.norm(torch.stack([p.grad.norm(p=2) for p,*_ in self.params()]), p=2)
    
    @torch.no_grad()
    def first_step(self):
        scale = self.rho / (self._grad_norm() + self.eps)
        for p,*_ in self.params():
            self.state[p]["e_w"] = e_w = p.grad * scale
            p.add_(e_w)  # climb to the local maximum "w + e(w)"
        if self.zero_grad: self.learn.opt.zero_grad()
        
    @torch.no_grad()    
    def second_step(self):
        for p,*_ in self.params(): p.sub_(self.state[p]["e_w"])

    def before_step(self, **kwargs):
        self.first_step()
        self.learn.pred = self.model(*self.xb); self.learn('after_pred')
        self.loss_func(self.learn.pred, *self.yb).backward()
        self.second_step()

In [ ]:
learn=Learner(dls,model=model,loss_func=LabelSmoothingCrossEntropy(),metrics=accuracy).to_non_native_fp16()

In [ ]:
learn.fit_one_cycle(1,1e-4,cbs=[SAM])

epoch,train_loss,valid_loss,accuracy,time


[]


RuntimeError: stack expects a non-empty TensorList

In [ ]:
learn.fit_one_cycle(200,1e-4,cbs=[CSVLogger(fname='medres10max.csv',append=False),SaveModelCallback(monitor='accuracy',fname='medres10max')])

epoch,train_loss,valid_loss,accuracy,time
0,2.208840,0.992757,0.556075,07:51
1,0.922655,0.741545,0.570093,07:50
2,0.699539,0.685949,0.602804,07:50
3,0.683380,0.701407,0.588785,07:50
4,0.666085,0.662352,0.598131,07:50
5,0.648569,0.668125,0.602804,07:50
6,0.630436,0.644596,0.616822,07:50
7,0.612507,0.648703,0.658879,07:50
8,0.616883,0.626004,0.663551,07:50
9,0.590286,0.644386,0.654206,07:50


Better model found at epoch 0 with accuracy value: 0.5560747385025024.
Better model found at epoch 1 with accuracy value: 0.5700934529304504.
Better model found at epoch 2 with accuracy value: 0.6028037667274475.
Better model found at epoch 6 with accuracy value: 0.6168224215507507.
Better model found at epoch 7 with accuracy value: 0.65887850522995.
Better model found at epoch 8 with accuracy value: 0.663551390171051.
Better model found at epoch 11 with accuracy value: 0.7242990732192993.
Better model found at epoch 12 with accuracy value: 0.7336448431015015.
Better model found at epoch 17 with accuracy value: 0.7570093274116516.
